# Data Cleaning

In [1]:
# Plots
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("notebook")
sns.set_style("white")

# Utilities
import numpy as np
import pandas as pd

# Shapefiles
import shapefile as shp

### Introducing the data sets

##### Data set 1: NYC poverty measure
Here we look at the income, ethnicity, age, poverty measures and other parameters of individuals in NYC.

In [2]:
focus_columns_nyc_data=(["SERIALNO", "AGEP","SEX", "NYCgov_MOOP","Boro","HIUnit_ID",
                "Ethnicity","PWGTP",
               "NYCgov_Income","NYCgov_Threshold","NYCgov_Pov_Stat", "NYCgov_EITC",
               "NYCgov_PovGap","NYCgov_PovGapIndex"])

ethnicities = {
    1: "Non-Hispanic White Individual ACS",
    2: "Non-Hispanic Black" ,
    3: "Non-Hispanic Asian" ,
    4: "Hispanic, Any Race",
    5: "Other Race/Ethnic Group"
}

df_nyc_poverty= pd.read_csv("data/NYCgov_Poverty_Measure_Data__2017_.csv")
df_nyc_poverty=df_nyc_poverty[focus_columns_nyc_data]

#add ethnicity
df_nyc_poverty=df_nyc_poverty.assign(ethnicity_name=df_nyc_poverty["Ethnicity"])

for key, value in ethnicities.items():
    df_nyc_poverty.loc[(df_nyc_poverty["ethnicity_name"] == key), "ethnicity_name"]=value

In [3]:
df_nyc_poverty.head(3)

,SERIALNO,AGEP,SEX,NYCgov_MOOP,Boro,HIUnit_ID,Ethnicity,PWGTP,NYCgov_Income,NYCgov_Threshold,NYCgov_Pov_Stat,NYCgov_EITC,NYCgov_PovGap,NYCgov_PovGapIndex,ethnicity_name
0,68,31,1,4673.138,2,1,1,56,80489.748482,15554.746967,2,0.0,0.0,0.0,Non-Hispanic White Individual ACS
1,68,28,2,1774.481,2,2,1,58,29917.996764,15554.746967,2,0.0,0.0,0.0,Non-Hispanic White Individual ACS
2,85,54,1,5500.131,3,1,1,111,61992.182430,21932.193223,2,0.0,0.0,0.0,Non-Hispanic White Individual ACS


##### Data set 2: p36, personal data (p) and New York (state 36)

The American Community Survey (ACS) Public Use Microdata Sample (PUMS) files are a set of untabulated records about individual people or housing units. The Census Bureau produces the PUMS files so that data users can create custom tables that are not available through pretabulated (or summary) ACS data products.
https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMS_Data_Dictionary_2017.pdf

The data set is only used to get the PUMA code of each individual in the NYC poverty data set


In [4]:
focus_columns_pums=(["SERIALNO","PUMA"])

df_pums_people = pd.read_csv("data/psam_p36.csv")
df_pums_people=df_pums_people[focus_columns_pums]

# This conversion allow us to merge NYC poverty measure and PUMS data set
df_pums_people["SERIALNO"] = df_pums_people["SERIALNO"]-2017000000000

In [5]:
df_pums_people.head(2)

,SERIALNO,PUMA
0,50,902
1,50,902


##### Data set 3: 2010_PUMA_Names

This dataset comes from https://www.census.gov/programs-surveys/geography/guidance/geo-areas/pumas.html. It is used for visualizing the names of each PUMA in the plotting of the household income.

In [6]:
df_puma_names = pd.read_csv("data/2010_PUMA_Names.csv")

In [7]:
df_puma_names.head(2)

,STATEFP,PUMA5CE,PUMA NAME
0,1,100,"Lauderdale, Colbert, Franklin & Marion (Northe..."
1,1,200,Limestone & Madison (Outer) Counties--Huntsvil...


##### Data set 4: PUMA shapefiles

This is the eventual data for the PUMAs visualization. https://catalog.data.gov/dataset/tiger-line-shapefile-2014-series-information-for-the-2010-census-public-use-microdata-area-puma

In [8]:
shp_path = "data/puma_shapefiles/tl_2014_36_puma10.shp"
sf = shp.Reader(shp_path)
records = pd.DataFrame(sf.records())

In [9]:
records.head(3)

,0,1,2,3,4,5,6,7,8,9
0,36,03803,3603803,NYC-Manhattan Community District 10--Central H...,G6120,S,3703008,241237,+40.8137240,-073.9429990
1,36,03707,3603707,NYC-Bronx Community District 5--Morris Heights...,G6120,S,3911379,24513,+40.8528801,-073.9087693
2,36,03201,3603201,Nassau County (Northwest)--North Hempstead Tow...,G6120,S,99590465,40464509,+40.8170441,-073.6919814


##### Data set 5: COVID 19 NYC state testing

This data comes from the official NYS health site, https://health.data.ny.gov/Health/New-York-State-Statewide-COVID-19-Testing/xdss-u53e. It is used to show number of positives each day and will be used for a temporal visualization of the COVID-19.

In [10]:
df_newyork = pd.read_csv("data/New_York_State_Statewide_COVID-19_Testing.csv")
df_newyork["Test Date"] = pd.to_datetime(df_newyork["Test Date"], format = "%m/%d/%Y")

In [11]:
county_to_borough = {
    "New York": "Manhattan",
    "Bronx": "Bronx",
    "Richmond": "Staten Island",
    "Queens": "Queens",
    "Kings": "Brooklyn"
}
df_nyc_covid = df_newyork[df_newyork["County"].isin(county_to_borough.keys())]

In [12]:
for key, value in county_to_borough.items():
    df_nyc_covid.loc[:,"County"]= df_nyc_covid["County"].replace(key, value)

C:\Users\Christian\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [13]:
df_nyc_covid

,Test Date,County,New Positives,Cumulative Number of Positives,Total Number of Tests Performed,Cumulative Number of Tests Performed
2,2020-04-13,Bronx,1061,23266,2294,45492
23,2020-04-13,Brooklyn,902,29306,2055,57245
30,2020-04-13,Manhattan,379,14880,1206,36449
40,2020-04-13,Queens,1068,35053,2430,63845
42,2020-04-13,Staten Island,292,7960,725,18130
...,...,...,...,...,...,...
2606,2020-03-02,Bronx,0,0,0,0
2627,2020-03-02,Brooklyn,0,0,0,0
2634,2020-03-02,Manhattan,0,0,0,0
2644,2020-03-02,Queens,0,0,0,0


##### Data set 6: COVID-19 zip-code data

This is ZIP-code data and is used to have better prediction of the actual correlation between poverty, demographics and covid infection. https://github.com/nychealth/coronavirus-data/blob/master/tests-by-zcta.csv

In [14]:
df_tests_zcta = pd.read_csv("data/tests-by-zcta.csv")
df_tests_zcta=df_tests_zcta.sort_values("zcta_cum.perc_pos")

# we drop entries with NaN values, the NA values have no ZCTA code and convert to object
df_tests_zcta=df_tests_zcta.dropna()
df_tests_zcta["MODZCTA"]=df_tests_zcta["MODZCTA"].astype("int").astype("str")
df_tests_zcta.columns=[i.lower() for i in list(df_tests_zcta.columns)]
df_tests_zcta.rename(columns={"total":"total tests"}, inplace=True)

In [15]:
df_tests_zcta.head(3)

,modzcta,positive,total tests,zcta_cum.perc_pos
6,10006,11,44,25.00
7,10007,34,109,31.19
5,10005,39,117,33.33


##### Data set 7: zip-code data

Finally, we have data for each zip-code, used to merge with the zcta data

In [16]:
datasets_n = 3
data_path = r"C:\Users\Christian\Dropbox\10. Semester\Social Data Analyse og Visualisering\Project\newyork_covid19_demographics\data_zcta"+"\\"
name_list = ["demographic", "economic", "social"] #"housing", didn't find much interesting stuff in those two
dataset_list=[]
df_temp_cat=[]
for name in name_list:
    for i in range(datasets_n):
        df_temp = pd.read_csv(data_path+name+str(i+1)+'.csv')
        df_temp=df_temp.loc[:, ~df_temp.columns.str.contains('^Unnamed')] #remove MOE and percent (specified as unnamed in DF)
        df_temp=df_temp.T
        df_temp.columns=[str(i).rstrip(" ").lstrip(" ").lower() for i in df_temp.iloc[0]]
        df_temp=df_temp.drop(df_temp.index[0]) 
        if(name == "demographic"):
            df2=df_temp[["total population",
                       'under 5 years', '5 to 9 years', '10 to 14 years', '15 to 19 years',
                       '20 to 24 years', '25 to 34 years', '35 to 44 years', '45 to 54 years',
                       '55 to 59 years', '60 to 64 years', '65 to 74 years', '75 to 84 years', "male","female",'median age in years'
                        ]]
            df1=df_temp.iloc[:,[55,57, 58, 60]]
            df_temp=pd.concat([df2,df1],axis=1)
        elif(name == "economic"):
            df1=df_temp[['total households',
                         'median household income',
                         'mean household income','persons under 18 in poverty',
                         'persons aged 18 to 64 in poverty',
                         'persons over 65 in poverty','worked at home','public transportation (excluding taxicab)']]
            df2=df_temp.iloc[:,[3,4,5,6,7,8,9,10,11,12,13]]
            df_temp=pd.concat([df2,df1],axis=1)
        elif(name == "social"):
            df_temp=df_temp[['less than 9th grade',
                         'high school graduate (includes equivalency)',
                         'bachelors degree',
                         'graduate or professional degree']] #this is data for age over 25
        df_temp = df_temp.T
        df_temp_cat.append(df_temp)
        
    df_temp_cat = pd.concat(df_temp_cat, axis=1)
    dataset_list.append(df_temp_cat)
    df_temp_cat=[]
    
df=pd.concat(dataset_list)
df.columns=[i.lstrip("ZCTA5 ") for i in df.columns]
#transpose the data
df=df.T
#remove duplicate columns
df = df.loc[:,~df.columns.duplicated()]
df.index.name = "zcta"
df.reset_index(level=0, inplace=True)
df=df.replace("-", 0)

### Preprocessing the data

First let's look at the size of the data sets

In [19]:
mb=round(df_nyc_poverty.memory_usage(index=True, deep=True).sum()*10**(-6),2)
print("df_nyc_poverty:")
print("Shape of data set (rows x columns): {}, dataframe megabyte usage: {}."
      .format(df_nyc_poverty.shape,mb))

mb=round(df_pums_people.memory_usage(index=True, deep=True).sum()*10**(-6),2)
print("df_pums_people:")
print("Shape of data set (rows x columns): {}, dataframe megabyte usage: {}."
      .format(df_pums_people.shape,mb))

mb=round(df_puma_names.memory_usage(index=True, deep=True).sum()*10**(-6),2)
print("df_puma_names:")
print("Shape of data set (rows x columns): {}, dataframe megabyte usage: {}."
      .format(df_puma_names.shape,mb))

mb=round(records.memory_usage(index=True, deep=True).sum()*10**(-6),2)
print("records:")
print("Shape of data set (rows x columns): {}, dataframe megabyte usage: {}."
      .format(records.shape,mb))

mb=round(df_tests_zcta.memory_usage(index=True, deep=True).sum()*10**(-6),2)
print("df_tests_zcta:")
print("Shape of data set (rows x columns): {}, dataframe megabyte usage: {}."
      .format(df_tests_zcta.shape,mb))

mb=round(df.memory_usage(index=True, deep=True).sum()*10**(-6),2)
print("df:")
print("Shape of data set (rows x columns): {}, dataframe megabyte usage: {}."
      .format(df.shape,mb))

df_nyc_poverty:
Shape of data set (rows x columns): (68094, 15), dataframe megabyte usage: 16.46.
df_pums_people:
Shape of data set (rows x columns): (196585, 2), dataframe megabyte usage: 3.15.
df_puma_names:
Shape of data set (rows x columns): (2380, 3), dataframe megabyte usage: 0.3.
records:
Shape of data set (rows x columns): (145, 10), dataframe megabyte usage: 0.08.
df_tests_zcta:
Shape of data set (rows x columns): (177, 4), dataframe megabyte usage: 0.02.
df:
Shape of data set (rows x columns): (177, 43), dataframe megabyte usage: 0.46.


Currently, we have 6 data sets, to narrow the amount of data sets down, making it easier to work with the data, some will be merged. `df_nyc_poverty`, `df_pums_people`, `df_puma_names` will be merged.

In [20]:
# Only looking at state 36 (NY state)
NY_STATEFP = 36
df_puma_names = df_puma_names[df_puma_names["STATEFP"] == NY_STATEFP]

#joining the pums people and puma names data sets.
df_puma_pmn = pd.merge(left=df_pums_people,right=df_puma_names, how='left', left_on="PUMA", right_on = "PUMA5CE")
df_puma_pmn=df_puma_pmn.drop(["STATEFP", "PUMA5CE"], axis=1).drop_duplicates(keep='first', inplace=False)

# merged the nyc poverty data and puma codes
df_poverty=pd.merge(left=df_nyc_poverty,right=df_puma_pmn, how='left', left_on="SERIALNO", right_on = "SERIALNO")

#Add borough names
df_poverty=df_poverty.assign(puma_names = [i.lstrip("NYC-").split(" Community")[0] for i in df_poverty["PUMA NAME"]])

Next we merge `df_tests_zcta` and `df` 

In [43]:
df_zcta = pd.merge(left=df, right=df_tests_zcta, how='left', left_on="zcta", right_on = "modzcta").drop("modzcta", axis=1)
df_zcta[df_zcta.columns[1:]]=df_zcta[df_zcta.columns[1:]].apply(pd.to_numeric, errors='coerce')
#other races
other_races = df_zcta['total population']-(df_zcta['hispanic or latino of any race']+ df_zcta['white alone']+df_zcta['black or african american alone']+ df_zcta['asian alone'])
df_zcta=df_zcta.assign(some_other_race=other_races)

We only look at relevant boroughs for new york covid data set

Checking NAs:

In [39]:
df_zcta.isna().sum().sum()+df_poverty.isna().sum().sum()+records.isna().sum().sum()+df_nyc_covid.isna().sum().sum()

0

Some useful dictionairies.

In [ ]:
boroughs = {
    1: "Bronx",
    2: "Brooklyn",
    3: "Manhattan",
    4: "Queens",
    5: "Staten Island"
}

population_boroughs = {
    "Bronx": 1384580,
    "Brooklyn": 2504721,
    "Manhattan": 1586381,
    "Queens": 2230619,
    "Staten Island": 468730
}

## Descriptive Statistics

In [22]:
from bokeh.plotting import *
from bokeh.models import ColumnDataSource, FactorRange, Legend, HoverTool, CustomJS, Slider, LinearAxis
from bokeh.models.widgets import Select
from bokeh.palettes import Category20, Spectral4,Spectral7,Category10
from bokeh.transform import dodge
from bokeh.palettes import Spectral4,Spectral7
from bokeh.layouts import widgetbox,row,column
output_notebook()

Loading BokehJS ...

### Model

In [45]:
df_zcta.columns

Index(['zcta', 'total population', 'under 5 years', '5 to 9 years',
       '10 to 14 years', '15 to 19 years', '20 to 24 years', '25 to 34 years',
       '35 to 44 years', '45 to 54 years', '55 to 59 years', '60 to 64 years',
       '65 to 74 years', '75 to 84 years', 'male', 'female',
       'median age in years', 'hispanic or latino of any race', 'white alone',
       'black or african american alone', 'asian alone', 'total households',
       'less than $10,000', '$10,000 to $14,999', '$15,000 to $24,999',
       '$25,000 to $34,999', '$35,000 to $49,999', '$50,000 to $74,999',
       '$75,000 to $99,999', '$100,000 to $149,999', '$150,000 to $199,999',
       '$200,000 or more', 'median household income', 'mean household income',
       'persons under 18 in poverty', 'persons aged 18 to 64 in poverty',
       'persons over 65 in poverty', 'worked at home',
       'public transportation (excluding taxicab)', 'less than 9th grade',
       'high school graduate (includes equivalency)'

0       134
1       267
2       375
3       308
4       368
       ... 
172    2166
173    1929
174    1267
175     411
176    1244
Length: 177, dtype: int64